# YuMi inverse kinematics sandbox

This notebook sets poses to be tracked by YuMi's inverse kinematics controller (running on a C++ ROS node).

What happens here:
* A ROS node is created using rospy
* 3 publishers are created to advertise on 3 different topics:
    * */yumi/joint_solver*: **topic** to advertise how the C++ node will "solve" for a control command in joint space. Takes a string with "IK" or "homing"
    * */yumi/left_pose_ref*: advertise the desired left end-effector pose
    * */yumi/right_pose_ref*: advertise the desired right end-effector pose

## Configuration

In [1]:
import rospy
import numpy as np
from std_msgs.msg import String
from geometry_msgs.msg import Pose

### Useful functions

In [2]:
# Convert list or np arrays to pose msg
def setpose_msg(position, quaternion):
    pose_msg = Pose()
    pose_msg.position.x = position[0]
    pose_msg.position.y = position[1]
    pose_msg.position.z = position[2]
    pose_msg.orientation.x = quaternion[0]
    pose_msg.orientation.y = quaternion[1]
    pose_msg.orientation.z = quaternion[2]
    pose_msg.orientation.w = quaternion[3]
    return pose_msg

### ROS-related initalizations

In [3]:
rospy.init_node('yumi_task_space_poses')
rate = rospy.Rate(250) # 10hz?

pub_ctrl = rospy.Publisher('/yumi/joint_solver', String, queue_size=10) # To advertise the control mode
pub_left = rospy.Publisher('/yumi/left_pose_ref', Pose, queue_size=10) # To advertise control references for left hand
pub_right = rospy.Publisher('/yumi/right_pose_ref', Pose, queue_size=10) # To advertise control references for right hand

### Start with homing the joints

In [4]:
joint_solver = 'homing'
pub_ctrl.publish(joint_solver)

## Moving the robot

### Left hand pose test

In [5]:
#Position
lpos = [0.3, 0.3, 0.3]

# Orientation
lquat = np.array([0, 0, 0, 1])      #x, y, z, w
lquat = lquat/np.linalg.norm(lquat)

# Publish
pub_left.publish(setpose_msg(lpos, lquat))
#rospy.loginfo(l_hand_pose)

### Right hand pose test

In [6]:
#Position
rpos = [0.3, -0.3, 0.3]

# Orientation
rquat = np.array([0, 0, 0, 1])      #x, y, z, w
rquat = rquat/np.linalg.norm(rquat)

# Publish
pub_right.publish(setpose_msg(rpos, rquat))
#rospy.loginfo(r_hand_pose)

### Set IK

In [7]:
joint_solver = 'pinv'
pub_ctrl.publish(joint_solver)

### Tracking trajectories

In [8]:
from IPython.display import clear_output

#### Sinusoidal waves

In [9]:
t = 0
pos_start = np.copy(rpos)
print pos_start

[ 0.3 -0.3  0.3]


In [14]:
while 1:
    t += 0.1
    #rpos[0] = pos_start[0] + 0.1*np.sin(0.1*t)
    rpos[1] = pos_start[1] + 0.1*np.cos(0.1*t)
    rpos[2] = pos_start[2] + 0.1*np.sin(0.1*t)
    #clear_output(wait=True)
    #print rpos[2]
    pub_right.publish(setpose_msg(rpos, rquat))
    rate.sleep()

KeyboardInterrupt: 

### Shutdown the node

In [ ]:
rospy.signal_shutdown('End of notebook')